# Trabajo sobre Redes Bayesianas

### Trabajo realizado por:

###### Francisco Javier Higueras Galván
###### Sergio Torrijos Campano

### Curso:

###### 16-17


# Proceso de Eliminación de Variables

En primer lugar, se lleva a cabo la lectura de los ficheros , que pueden estar en formato ___".bif"___ o en formato ___".xml"___ para mayor comodidad del usuario.

NOTA: Para cambiar el fichero según queramos xmlbif o bif deberemos cambiarlo en la casilla donde se declara la variable 'reader'. 

In [ ]:
import networkx  # Permite trabajar con grafos
import pgmpy.models as pgmm  # Modelos gráficos de probabilidad
import pgmpy.factors as pgmf  # Tablas de probabilidades condicionales y
                              # potenciales de probabilidad

#import pgmpy.factors.discrete.CPD as pgmf
import pgmpy.inference as pgmi  # Inferencia probabilística exacta
import pgmpy.readwrite.BIF as rwBIF

from pgmpy.readwrite import BIFReader
from pgmpy.readwrite import XMLBIFReader

    
#reader = XMLBIFReader("influenzaSmokes.xml")
reader = BIFReader("Modelo_accidente.bif")
Redes=reader.get_model()

Con el método ___"Descartar variables"___ , dada la red , el nodo al cual queremos calcular la probabilidad y una o más evidencias, nos devolverá la lista de variables que son descartadas por no ser antecesoras de ninguna variable de evidencia o consulta.

In [ ]:
def Descartar_variables(red,node,listaFijos):
    ancestros = set()
    listaFijosDefinitiva = listaFijos.copy()
    for nodes in node:
        ancestros.add(nodes)
    for lista in listaFijos:
     #   copia = listaFijos.copy()
     #   copia.remove(lista)
      #  if red.is_active_trail(node, lista, copia):
            ancestros.add(lista)
     #   else:
        #    listaFijosDeminitiva.remove(lista)
    for variable in ancestros.copy():
        ancestros |= networkx.ancestors(Redes, variable)
    VariablesDescartadas=set(red.nodes()) - ancestros
    print("Las variables descartadas son :")
    print(VariablesDescartadas)
    return ancestros

Con el método ___"Obtener Factores"___ conseguimos obtener todos los factores de la lista de variables obtenidas en el método anterior.

In [ ]:
def Obtener_Factores(red,listaVariables,listaFijas,listaCardinalidades):
    VariablesNoFijas=list(set(listaVariables) - set(listaFijas))
    ListaFactores={"Nombre":"Arturo Elias","Apellido":"Anton"}
    ListaFactores.clear()
    i=0
   
    for vnf in VariablesNoFijas:
        Tabla_CPD=red.get_cpds(vnf)
        phi_R = Tabla_CPD.to_factor()
        ListaFactores[vnf]=phi_R
    for lf in listaFijas:
        Tabla_CPD=red.get_cpds(lf)
        phi_S=Tabla_CPD.to_factor()
        phi_S.reduce([(lf, listaCardinalidades[i])])
        ListaFactores[lf]=phi_S
        i=i+1
    return ListaFactores

Con el método ___"Descartar Fijos"___, dadas las variables de evidencia y consulta, eliminamos las variables dadas del método de eliminación de variables si éstas son independientes del nodo que queremos consultar y las otras variables fijas.

In [ ]:
def Descartar_fijos(red,node,listaFijos):
    numeroListaFijos=len(listaFijos)
    listaFijosProvisional=listaFijos.copy()
    for nodo in node:
        elnodo=nodo
    while numeroListaFijos<=len(listaFijos):
        numeroListaFijos=len(listaFijosProvisional)
        listaFijosProvisional=listaFijos.copy()
        for fijo in listaFijosProvisional:
            listaFijos.remove(fijo)
            if len(listaFijos) <=0:
                 dependientes=red.active_trail_nodes(elnodo)
            else:
                 dependientes=red.active_trail_nodes(elnodo,listaFijos)
            if fijo not in dependientes:
                break
            else:
                listaFijos.append(fijo)
        if numeroListaFijos == len(listaFijos):
            numeroListaFijos=numeroListaFijos + 1
    return listaFijos

El método ___"Min Degree Heuristica"___, una lista de variables ordenadas según el orden que hay que seguir para su eliminación siguiendo la heurística: ___MIN DEGREE___

In [ ]:
def Min_Degree_Heuristica(ListaFactores,node):
    listaFactoresOrdenada=list()
    for nodo in node:
        del ListaFactores[nodo]
    numeroIncluidos=len(ListaFactores)
    minimo=0
    minimoIncluido=numeroIncluidos
    while numeroIncluidos!=0:
        for key,value in ListaFactores.items():
            if len(value.scope())==minimo:
                listaFactoresOrdenada.append(key)
                del ListaFactores[key]
                numeroIncluidos=numeroIncluidos-1
                break
            else:
                continue
        if minimoIncluido==numeroIncluidos:
            minimo=minimo+1
        else:
            minimoIncluido=numeroIncluidos
    return listaFactoresOrdenada

El método ___"recomendador"___ nos devuelve , una vez finalizada la consulta, una recomendación para realizar una nueva consulta con unas variables de evidencia distintas a las seleccionadas inicialmente, que nos aportaran unos resultados más optimos para la consulta que estamos realizando.
En caso de no haber variables de evidencia que nos aporten mejores resultados de los obtenidos, se le comunicará de igual manera al usuario.

In [ ]:
def recomendador(red,atributo1,atributo2):
    result=list()
    ancestrillos= networkx.ancestors(red, atributo1)
    
    if not ancestrillos:
        print("Usted ha escogido la mejor elección de evidencias")
        
    else :
        for i in ancestrillos:
            if atributo1 in red.neighbors(i) and i not in atributo2:
                result.append(i)
    
    return result

El método ___"eliminar Variables"___ dada la lista de factores obtenida en un método anterior, y la lista de variables ordenadas según la heurística deseada por el usuario para obtener un orden en la eliminación de las mismas, obtenemos finalmente el resultado obtenido de la consulta previamente realizada

In [ ]:
def eliminar_variables(red,listaFactoresOrdenados,listaFactores2,nodes):
    phi_noT='vacio'
    for factorillo in listaFactoresOrdenados:
        listaFactorado=set()
        listaVariables=set()


        for listaFactorcillo in list(listaFactores2.keys()):
            if factorillo in listaFactores2[listaFactorcillo].scope():
                listaFactorado.add(listaFactores2[listaFactorcillo])
                listaVariables.add(listaFactorcillo)
            else :
                continue
        if len(listaFactorado)==0:
            continue
        else:
            phi_noT=pgmf.factor_product(*(i for i in listaFactorado))
            phi_noT.marginalize([factorillo])
            for listaFactorcilleto in listaVariables:
                listaFactores2[listaFactorcilleto]=phi_noT
            print(factorillo)
            print(phi_noT.scope())
            print(phi_noT)
    for nodo in nodes:
        node = nodo
    if phi_noT == 'vacio':
        phi = pgmf.factor_product(listaFactores2[node])
    else:
        phi = pgmf.factor_product(phi_noT,listaFactores2[node])
    phi.normalize()
    return phi

El método ___"Eliminacion de Variables"___ auna todos los métodos anteriormente vistos en uno solo, haciendo así que el usuario unicamente tenga que seleccionar la probabilidad que desea hallar y dicho método facilitará al usuario el resultado de la consulta.

In [ ]:
def eliminacion_de_variables(red):
    atri1 = set()
    atri2 = list()
    atri3 = list()
    print("Calcular la probabilidad de:")
    atributo1 = input()
    while (atributo1 not in red.nodes()) or atributo1 =='':
        print("Por favor, introduzca un valor válido.")
        print("Calcular la probabilidad de:")
        atributo1 = input()
    atri1.add(atributo1)
    print("Dado:")
    atributo2 = input()
    print("¿Es Cierto o Falso?(si es cierto es 1 si es falso es 0, si no es booleana introduzca su valor)")
    atributo3 = input()
    while atributo2 not in red.nodes():
        print("Por favor, introduzca un valor válido.")
        print("Dado:")
        atributo2 = input()
        print("¿Es Cierto o Falso?(si es cierto es 1 si es falso es 0, si no es booleana introduzca su valor)")
        atributo3 = input()
    atri2.append(atributo2)
    atri3.append(int(atributo3))
    while atributo2 != '':
        print("y:")
        atributo2 = input()
        if atributo2 == '':
            break
        if atributo2 not in red.nodes() :
            print("Por favor, introduzca un valor válido.")
        else: 
            print("¿Es Cierto o Falso?(si es cierto es 1 si es falso es 0, si no es booleana introduzca su valor)")
            atributo3 = input()
            atri2.append(atributo2)
            atri3.append(int(atributo3))


    listaFijos=Descartar_fijos(red,atri1,atri2)
    listaVariables=Descartar_variables(red,atri1,listaFijos)
    print("Las variables que se tendran en cuenta son:")
    print(listaVariables)
    listaFactores=Obtener_Factores(red,listaVariables,listaFijos,atri3)
    
    listaFactoresOrdenados=Min_Degree_Heuristica(listaFactores,atri1)
    print("Orden en el que se van a eliminar los factores:")
    print(listaFactoresOrdenados)
    listaFactores2=Obtener_Factores(red,listaVariables,listaFijos,atri3)

    for casol in listaFactoresOrdenados.copy():
        ancestros = networkx.descendants(red, casol)
        if len(ancestros)==0:
            listaFactoresOrdenados.remove(casol)
        else:
            continue
    phi=eliminar_variables(red,listaFactoresOrdenados,listaFactores2,atri1)
    print("Resultado final:")
    print(phi)
    
    print("Se recomienda elegir las siguientes evidencias, ya que podrían modificar mucho su resultado:")
    recomienda=recomendador(red,atributo1,atri2)
    print(recomienda)
    return phi

# Eliminación de Variables.

Aquí el usuario realizará la consulta que desee.

__NOTA:__ Se realizará la consulta al modelo leído en el apartado 1.

In [ ]:
eliminacion_de_variables(Redes)

# Proceso de Muestreo por Rechazo

La ___"Muestra Aleatoria"___ es una asignación de valores a las variables de una red bayesiana, con probabilidad de generación igual a la probabilidad conjunta de que las variables tomen esos valores.

In [ ]:
import random

def seleccionar_probabilidad(cpd, valor, evidencia):
    padres = cpd.evidence if cpd.evidence else []
    valores_evidencia = tuple(evidencia[var] for var in padres)
    if 'Sangre' in Redes.nodes():
        return cpd.values[valor][valores_evidencia]
    else:
        return cpd.values[valores_evidencia][valor]

def generar_valor_aleatorio(cardinalidad, probabilidades):
    p = random.random()
    acumuladas = 0
    for valor in range(cardinalidad):
        acumuladas += probabilidades[valor]
        if  p<=acumuladas:
            return valor
    
        
def muestra_aleatoria(red):
    variables = networkx.topological_sort(red)
    muestra={}
    for variable in variables:
        tabla= red.get_cpds(variable)
        cardinalidad= tabla.variable_card
        probabilidades = [seleccionar_probabilidad(tabla,i,muestra) 
                     for i in range(cardinalidad)]
        valor = generar_valor_aleatorio(cardinalidad,probabilidades)
        muestra[variable]=valor
    return muestra


El método ___"Muestreo con rechazo"___ dada una red bayesiana, la consulta, una o más evidencias y un número "N", se realizarán "N" muestras aleatorias, las cuales se analizarán y seguidamente se dará una estimación de la probabilidad de que ocurra el hecho dado por el usuario con las evidencias aportadas por él.

In [ ]:
import itertools

def muestreo_con_rechazo(red,consulta,evidencia,N):
    muestras = [muestra_aleatoria(red) for i in range(N)]
    muestrasValidas = []
    cardinalidades = red.get_cpds(consulta).variable_card
    combinaciones=list(range(0,cardinalidades))
    for muestra in muestras:
        if all(muestra[variable] == evidencia[variable] for variable in evidencia):
            muestrasValidas.append(muestra)
    frecuencias = {combinacion : 0 for combinacion in combinaciones}
    for muestrad in muestrasValidas:
        combinacion = muestrad[consulta]
        if combinacion==None:
            combinacion=1
        frecuencias[combinacion] +=1
    print(frecuencias)
    Modelo_ejemplo = pgmm.BayesianModel()
    Modelo_ejemplo.add_node(consulta)
    Ejemplo_CPD = pgmf.TabularCPD(consulta, cardinalidades, [[frecuencias[key] for key in frecuencias.keys()]])
    print(Ejemplo_CPD)
    Ejemplo_CPD.normalize()
    return Ejemplo_CPD



# Muestreo Por Rechazo

In [ ]:
def Rechazo(red):
    
    atri11 = ""
    atri21 = list()
    atri31 = list()
    atri41 = 0
    print("Calcular la probabilidad de:")
    atributo11 = input()
    while (atributo11 not in red.nodes()) or atributo11 =='':
        print("Por favor, introduzca un valor válido.")
        print("Calcular la probabilidad de:")
        atributo11 = input()
    atri11 = atributo11
    print("Dado:")
    atributo21 = input()
    print("¿Es Cierto o Falso?(si es cierto es 1 si es falso es 0, si no es booleana introduzca su valor)")
    atributo31 = input()
    while atributo21 not in red.nodes():
        print("Por favor, introduzca un valor válido.")
        print("Dado:")
        atributo21 = input()
        print("¿Es Cierto o Falso?(si es cierto es 1 si es falso es 0, si no es booleana introduzca su valor)")
        atributo31 = input()
    atri21.append(atributo21)
    atri31.append(int(atributo31))
    while atributo21 != '':
        print("y:")
        atributo21 = input()
        if atributo21 == '':
            break
        if atributo21 not in red.nodes() :
            print("Por favor, introduzca un valor válido.")
        else: 
            print("¿Es Cierto o Falso?(si es cierto es 1 si es falso es 0, si no es booleana introduzca su valor)")
            atributo31 = input()
            atri21.append(atributo21)
            atri31.append(int(atributo31))
            
    print("¿Cuantas muestras aleatorias desea generar:")
    atributo41 = input()
    while (atributo41 == 0 or atributo41 ==''):
        print("Por favor, introduzca un valor válido.")
        print("¿Cuantas muestras aleatorias desea generar?")
        atributo41 = input()
    atri41 = (int(atributo41))   
            
    ListaCardinalidades={"Nombre":"Arturo Elias","Apellido":"Anton"}
    ListaCardinalidades.clear()
    i=0
   
    for lf in atri21:
        ListaCardinalidades[lf]=atri31[i]
        i=i+1

    muestreo =muestreo_con_rechazo(red, atri11 , ListaCardinalidades, atri41)
    print(muestreo)

In [ ]:
Rechazo(Redes)

Rechazo(Redes)